In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling,pipeline
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training,PeftConfig
from trl import SFTTrainer
import torch

/home/qvr905/.conda/envs/phi3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [2]:
from datasets import load_dataset
from random import randrange


dataset = load_dataset("flytech/python-codes-25k", split = "train")
dataset = dataset.train_test_split(test_size=0.1, shuffle=True)
#dataset = dataset.map(formatting_prompts_func, batched = True,)
print(len(dataset["train"]))
print(len(dataset["test"]))

for i in range(5):
    print(dataset["train"][i])

print(f"dataset size: {len(dataset['train'])}")



44663
4963
{'instruction': 'Analyze a given list of numbers and print out the mean and variance using Python numbers = [2, 4, 6, 9, 12, 13]', 'input': '', 'text': 'Analyze a given list of numbers and print out the mean and variance using Python numbers = [2, 4, 6, 9, 12, 13] Let\'s get cracking! Time to get busy! ```python\ndef analyze_list(numbers):\n mean = 0\n for num in numbers:\n mean += num\n mean /= len(numbers)\n \n variance = 0\n for num in numbers:\n variance += (num - mean)**2\n variance /= len(numbers)\n \n print("Mean:", mean)\n print("Variance:", variance)\n```', 'output': '```python\ndef analyze_list(numbers):\n mean = 0\n for num in numbers:\n mean += num\n mean /= len(numbers)\n \n variance = 0\n for num in numbers:\n variance += (num - mean)**2\n variance /= len(numbers)\n \n print("Mean:", mean)\n print("Variance:", variance)\n```'}
{'instruction': 'Compare two strings in Python to determine if they are anagrams of one another str1 = "abcd"\nstr2 = "dcba"', 'input': 

In [3]:
base_model_id = "microsoft/phi-2"

#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_eos_token=True, use_fast=True, max_length=250)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

compute_dtype = getattr(torch, "float16") #change to bfloat16 if are using an Ampere (or more recent) GPU
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          base_model_id, trust_remote_code=True, quantization_config=bnb_config, revision="refs/pr/23", device_map={"": 0}, torch_dtype="auto", flash_attn=True, flash_rotary=True, fused_dense=True
)

model = prepare_model_for_kbit_training(model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.42s/it]
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [4]:
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["Wqkv", "out_proj"])

training_arguments = TrainingArguments(
        output_dir="./phi2-results3",
        save_strategy="epoch",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=12,
        log_level="debug",
        save_steps=100,
        logging_steps=25,
        learning_rate=1e-4,
        eval_steps=50,
        optim='paged_adamw_8bit',
        fp16=True, #change to bf16 if are using an Ampere GPU
        num_train_epochs=1,
        max_steps=1000,
        warmup_steps=100,
        lr_scheduler_type="linear",
        seed=42)

In [5]:
#Train
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset["train"],
        #eval_dataset=dataset['test'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=1024,
        tokenizer=tokenizer,
        args=training_arguments,
        packing=False
)


Map: 100%|██████████| 44663/44663 [00:07<00:00, 5664.65 examples/s]
/home/qvr905/.conda/envs/phi3/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend


In [6]:

trainer.train()

Currently training with a batch size of: 4
***** Running training *****
  Num examples = 44,663
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 12
  Total optimization steps = 1,000
  Number of trainable parameters = 7,864,320


Step,Training Loss
25,1.294400
50,1.221400
75,1.115600
100,1.007500
125,0.928500
150,0.887000
175,0.843800
200,0.827700
225,0.782100
250,0.780400


Saving model checkpoint to ./phi2-results3/checkpoint-930
loading configuration file config.json from cache at /home/qvr905/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670/config.json
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
Model config PhiConfig {
  "_name_or_path": "microsoft/phi-2",
  "activation_function": "gelu_new",
  "architectures": [
    "PhiForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attn_pdrop": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/phi-2--configuration_phi.PhiConfig",
    "AutoModelForCausalLM": "microsoft/phi-2--modeling_phi.PhiForCausalLM"
  },
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "flash_attn": false,
  "flash_rotary": false,
  "fused_dense": false,
  "hidden_act": "gelu_new",
  "initializer_range": 0.02,
  "intermediate_size": 10240,
  "layer_norm_eps": 1e-05

TrainOutput(global_step=1000, training_loss=0.7929623985290527, metrics={'train_runtime': 6008.0707, 'train_samples_per_second': 7.989, 'train_steps_per_second': 0.166, 'total_flos': 1.9040979310239744e+17, 'train_loss': 0.7929623985290527, 'epoch': 1.07})

Saving model checkpoint to ./phi2-results2/tmp-checkpoint-1000 \
tokenizer config file saved in ./phi2-results2/tmp-checkpoint-1000/tokenizer_config.json \
Special tokens file saved in ./phi2-results2/tmp-checkpoint-1000/special_tokens_map.json \

In [7]:
peft_model_id="phi_peft_ft2"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)


loading configuration file config.json from cache at /home/qvr905/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670/config.json
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
Model config PhiConfig {
  "_name_or_path": "microsoft/phi-2",
  "activation_function": "gelu_new",
  "architectures": [
    "PhiForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attn_pdrop": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/phi-2--configuration_phi.PhiConfig",
    "AutoModelForCausalLM": "microsoft/phi-2--modeling_phi.PhiForCausalLM"
  },
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "flash_attn": false,
  "flash_rotary": false,
  "fused_dense": false,
  "hidden_act": "gelu_new",
  "initializer_range": 0.02,
  "intermediate_size": 10240,
  "layer_norm_eps": 1e-05,
  "layer_norm_epsilon": 1e-05,
  "model_type": "phi-msft

('phi_peft_ft2/tokenizer_config.json',
 'phi_peft_ft2/special_tokens_map.json',
 'phi_peft_ft2/vocab.json',
 'phi_peft_ft2/merges.txt',
 'phi_peft_ft2/added_tokens.json',
 'phi_peft_ft2/tokenizer.json')

In [8]:
trainer.save_model("/work/qvr905/FineTune/phi/modelbk2")

Saving model checkpoint to /work/qvr905/FineTune/phi/modelbk2
loading configuration file config.json from cache at /home/qvr905/.cache/huggingface/hub/models--microsoft--phi-2/snapshots/b10c3eba545ad279e7208ee3a5d644566f001670/config.json
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
Model config PhiConfig {
  "_name_or_path": "microsoft/phi-2",
  "activation_function": "gelu_new",
  "architectures": [
    "PhiForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attn_pdrop": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/phi-2--configuration_phi.PhiConfig",
    "AutoModelForCausalLM": "microsoft/phi-2--modeling_phi.PhiForCausalLM"
  },
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "flash_attn": false,
  "flash_rotary": false,
  "fused_dense": false,
  "hidden_act": "gelu_new",
  "initializer_range": 0.02,
  "intermediate_size": 10240,
  "layer_norm_eps": 1

In [ ]:
# Load peft config for pre-trained checkpoint etc.
peft_model_id = "results"
config = PeftConfig.from_pretrained(peft_model_id)